In [46]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

In [47]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112475 entries, 0 to 112474
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   dteday        112475 non-null  object 
 1   hr            112475 non-null  float64
 2   casual        112475 non-null  int64  
 3   registered    112475 non-null  int64  
 4   temp_c        112475 non-null  float64
 5   feels_like_c  112475 non-null  float64
 6   hum           112475 non-null  float64
 7   windspeed     112475 non-null  float64
 8   weathersit    112475 non-null  int64  
 9   season        112475 non-null  int64  
 10  holiday       112475 non-null  int64  
 11  workingday    112475 non-null  int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 10.3+ MB


In [48]:
bikes['year_month'] = pd.to_datetime(bikes['dteday'], format='%m/%d/%Y').dt.to_period('M')
bikes.head()

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday,year_month
0,1/1/2011,0.0,3,13,3.0,3.0,0.7957,0.8,1,1,0,0,2011-01
1,1/1/2011,1.0,8,30,1.7,1.7,0.8272,0.8,1,1,0,0,2011-01
2,1/1/2011,2.0,5,26,1.9,1.9,0.8157,1.1,1,1,0,0,2011-01
3,1/1/2011,3.0,3,9,2.5,2.5,0.7831,0.8,1,1,0,0,2011-01
4,1/1/2011,4.0,0,1,2.0,2.0,0.8075,1.1,1,1,0,0,2011-01


In [49]:
bikes['day_of_week'] = pd.to_datetime(bikes['dteday'], format='%m/%d/%Y').dt.dayofweek
bikes.head()

,dteday,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday,year_month,day_of_week
0,1/1/2011,0.0,3,13,3.0,3.0,0.7957,0.8,1,1,0,0,2011-01,5
1,1/1/2011,1.0,8,30,1.7,1.7,0.8272,0.8,1,1,0,0,2011-01,5
2,1/1/2011,2.0,5,26,1.9,1.9,0.8157,1.1,1,1,0,0,2011-01,5
3,1/1/2011,3.0,3,9,2.5,2.5,0.7831,0.8,1,1,0,0,2011-01,5
4,1/1/2011,4.0,0,1,2.0,2.0,0.8075,1.1,1,1,0,0,2011-01,5


In [50]:
bikes['total_riders'] = bikes['casual'] + bikes['registered']
bikes['total_riders'].head()

,total_riders
0,16
1,38
2,31
3,12
4,1


In [51]:
bikes['month'] = pd.to_datetime(bikes['dteday'], format='%m/%d/%Y').dt.month
bikes['month'].head()

,month
0,1
1,1
2,1
3,1
4,1


In [52]:
bikes['hr_cos'] = bikes['hr'].apply(lambda x: np.cos(x * (2 * np.pi / 24)))
bikes['hr_sin'] = bikes['hr'].apply(lambda x: np.sin(x * (2 * np.pi / 24)))

In [53]:
X = bikes[['hr_cos','hr_sin', 'temp_c', 'year_month', 'day_of_week', 'day_of_week', 'temp_c', 'weathersit', 'month', 'holiday', 'hum', 'windspeed']]
X = pd.get_dummies(X, columns=['year_month', 'day_of_week', 'weathersit', 'month'])
y = bikes['total_riders']

In [54]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [56]:
from sklearn.preprocessing import MinMaxScaler

norm = MinMaxScaler().fit(X_train)

X_train = norm.transform(X_train)
X_test = norm.transform(X_test)

In [57]:
model = Sequential()

model.add(Dense(128, input_dim=len(X_train[0]), activation='swish'))
model.add(Dropout(.45))
model.add(Dense(256, activation='swish'))
model.add(Dense(64, activation='swish'))
model.add(Dense(32, activation='leaky_relu'))

model.add(Dense(1, activation='swish'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [58]:
opt = keras.optimizers.Adam()
model.compile(loss="mean_squared_error", optimizer=opt, metrics=['mse'])

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_mse', patience=30)

history = model.fit(X_train, y_train, epochs=1000, validation_split=.35, batch_size=265, callbacks=[early_stop],shuffle=False)
# history = model.fit(train_features, train_labels, epochs=2000, verbose=0, validation_split = .2, batch_size=tester2,
#                     callbacks=[early_stop, PrintDot()])

hist = pd.DataFrame(history.history)

Epoch 1/1000
194/194 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 29128.8242 - mse: 29128.8242 - val_loss: 23855.8242 - val_mse: 23855.8242
Epoch 2/1000
194/194 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 25968.2441 - mse: 25968.2441 - val_loss: 20663.0586 - val_mse: 20663.0586
Epoch 3/1000
194/194 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 23110.9297 - mse: 23110.9297 - val_loss: 18325.0742 - val_mse: 18325.0742
Epoch 4/1000
194/194 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 20867.6875 - mse: 20867.6875 - val_loss: 16997.5801 - val_mse: 16997.5801
Epoch 5/1000
194/194 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 19903.7012 - mse: 19903.6992 - val_loss: 15927.7109 - val_mse: 15927.7109
Epoch 6/1000
194/194 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 18592.0742 - mse: 18592.0742 - val_loss: 15251.9355 - val_mse: 15251.9355
Epoch 7/1000
194/194 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 17505.0176 - mse: 17505.0176 - val_loss: 14566.2881 - val_mse: 14566.2881
Epoch 8/1000
194/194 ━━━━━━━━━━━━━━━━━━━━ 2s

In [ ]:
model.summary()

In [ ]:
# h = hist
hist = hist.reset_index()
# h

In [ ]:
def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error ')
    plt.plot(hist['index'], hist['mse'], label='Train Error')
    plt.plot(hist['index'], hist['val_mse'], label = 'Val Error')
    plt.legend()
    # plt.ylim([0,50])

plot_history()

In [ ]:
predictions = np.round(model.predict(X_test),1)

In [ ]:
result = root_mean_squared_error(y_test, predictions)
result

In [ ]:
r2 = r2_score(y_test,predictions)
r2